In [63]:
import pandas as pd 
import numpy as np
from word2number import w2n
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Análisis exploratorio

En una fase previa analizamos en profundidad el archivo csv inicial, en la cual:
- Identificamos y nos familiarizamos con los datos a tratar y que posteriormente se tendrán que modificar.

- Diferenciamos los datos relevantes de los que no lo son. 

___

# Limpieza de datos

- En este apartado decidimos eliminar columnas basándonos en el tipo de información y la relevancia de los datos. 

Las columnas que se han eliminado son:
- **Employee Count:** Se trata de una columna prescindible, se puede contar el número de empleados por filas o por números de referencia de empleados.

- **Role Department:** La información de esta columna es el resultado de la fusión de dos columnas.

- **Number Children:** No hay valores

- **Over 18:** Columna sin sentido, está desactualizada. Se puede calcular con otras columnas.

- **Same As Month Income**: Es una columna repetida, una copia de Monthly Income

- **Salary:** La eliminamos por inconsistente, en todas las filas sale 1000000000$. Crearemos una nueva columna con el cálculo de los datos 

- **Standard Hours** Hay un elevado número de nulos y en las filas donde hay datos, en todos salen 80. Este dato no tiene sentido, decidimos eliminar la columna, teniendo en cuenta que hay otras columnas que hacen referencia al sueldo como son: Monthly Income, Monthly Rate, Hourly Rate, Daily Rate.

- **Years in Current Role:** Hay un elevado número de nulos y los datos son inconsistentes

In [64]:
df_ejemplo=pd.read_csv("HR RAW DATA.csv",index_col=0)

In [65]:
df_ejemplo.shape

(1614, 41)

In [66]:
#Homogeneizamos el nombre de las columnas 
nuevas_columnas = {columna: columna.lower() for columna in df_ejemplo.columns}

In [67]:
df_ejemplo.rename(columns=nuevas_columnas, inplace= True)

In [68]:
#Modificamos el nombre de la columna para que sea mas intuitivo
df_ejemplo.rename(columns = {"datebirth": "yearbirth"}, inplace = True )

In [69]:
# Eliminamos columnas que identificamos previamente

df_ejemplo.drop(['roledepartament','numberchildren','over18', 'sameasmonthlyincome','employeecount','salary','standardhours', 'yearsincurrentrole'],axis = 1, inplace=True)

In [70]:
df_ejemplo["age"]

0       51
1       52
2       42
3       47
4       46
        ..
1609    36
1610    45
1611    39
1612    36
1613    46
Name: age, Length: 1614, dtype: object

_______

# Transformación de datos I

- **Age:**  Está en formato object, lo pasamos a numérico con una función.

- Hacemos otra función para modificar las columnas **dailyrate**, **employeenumber**, **monthlyincome**, **performancerating**, **totalworkingyears**, **worklifebalance** y **distancefromhome**.
 
    **Los cambios que se aplicarán son:**
- Quitamos el símbolo '$' . Lo reemplazamos por ' ' 

- Reemplazamos ',' por  '.' para que Python pueda tratar correctamente los float

- Pasamos los valores numéricos que están en formato object a float

In [71]:
#Para columna de Age

def convertir_a_numero(valor):
    if isinstance(valor, int): # Verifica si el valor ya es un entero, si lo es no hace nada
        return valor
    elif isinstance(valor, str): # Intenta convertir el valor a número
        try: 
            numero = int(valor)
            return numero
        except ValueError:
            try: # aquí es donde se usa la función w2n en el try pq si la conversion falla entonces se usará
                numero = w2n.word_to_num(valor)
                return numero
            except ValueError:
                print(f"Error: No se pudo convertir '{valor}' a número.")
                return None
    else:
        print(f"Error: Tipo de dato no soportado para el valor '{valor}'.")
        return None


In [72]:
#lista con columnas a limpiar
columnas_limpiar = ['dailyrate','employeenumber','monthlyincome','performancerating','totalworkingyears','worklifebalance','distancefromhome']

In [73]:
df_ejemplo['age'] = df_ejemplo['age'].apply(convertir_a_numero) #llamamos a la función

In [74]:
def limpiar_datos (columna):

      try:
          df_ejemplo[columna] =(df_ejemplo[columna].str.replace('$', ''))
    
      except:
          pass
      
      try:
          df_ejemplo[columna] =(df_ejemplo[columna].str.replace(',', '.')) 
    
      except:
          pass
         
      try:
        df_ejemplo[columna]=pd.to_numeric(df_ejemplo[columna]).astype(float)
      except:
        pass
      return df_ejemplo[columna]

In [75]:
#Limpiamos las columnas de números para convertirlas a numéricas
for i in columnas_limpiar:
   df_ejemplo[i]=limpiar_datos(i)
   print(df_ejemplo[i])
   
 

0        684.0
1        699.0
2        532.0
3        359.0
4       1319.0
         ...  
1609     530.0
1610     805.0
1611     903.0
1612    1229.0
1613     566.0
Name: dailyrate, Length: 1614, dtype: object
0       162.0
1       259.0
2       319.0
3         NaN
4         NaN
        ...  
1609    967.0
1610    972.0
1611      NaN
1612    990.0
1613      NaN
Name: employeenumber, Length: 1614, dtype: float64
0       19537.0
1       19999.0
2       19232.0
3       17169.0
4           NaN
         ...   
1609    10325.0
1610     4447.0
1611        NaN
1612        NaN
1613    10845.0
Name: monthlyincome, Length: 1614, dtype: float64
0       3.0
1       3.0
2       3.0
3       3.0
4       3.0
       ... 
1609    NaN
1610    3.0
1611    3.0
1612    NaN
1613    3.0
Name: performancerating, Length: 1614, dtype: float64
0        NaN
1       34.0
2       22.0
3        NaN
4        NaN
        ... 
1609     NaN
1610     NaN
1611     9.0
1612    12.0
1613     NaN
Name: totalworkingyears, Lengt

_____

# Transformación de datos II

- Cambiamos de categoría la columna 'género'. Reemplazamos sus valores de 0 y 1 por Male y Female.

- Homogeneizamos la columna 'maritalstatus'.

- Modificamos los valores de la columna  'remotework', 0 y 1, True or False por Yes y No

- Modificamos los valores negativos de la columna DistanceFromHome a nulos para luego convertirlos a unknown en la gestión de nulos. 


In [76]:
df_ejemplo['gender'] = df_ejemplo['gender'].replace({0: 'male', 1: 'female'})
df_ejemplo['gender'].value_counts()

gender
male      971
female    643
Name: count, dtype: int64

In [77]:
df_ejemplo['maritalstatus']= df_ejemplo['maritalstatus'].replace({'divorced':'Divorced','Marreid':'Married'})

In [78]:
df_ejemplo['remotework'].value_counts()

remotework
1        360
True     345
0        309
False    305
Yes      295
Name: count, dtype: int64

In [79]:
df_ejemplo['remotework']= df_ejemplo['remotework'].replace({('1'):'Yes',('0') or ('False'): 'No', 'True': 'Yes', 'False': 'No'})

In [80]:
df_ejemplo['remotework'].value_counts()

remotework
Yes    1000
No      614
Name: count, dtype: int64

In [81]:

df_ejemplo["distancefromhome"] = df_ejemplo["distancefromhome"].apply(lambda x: np.nan if x < 0 else x)

**ELIMINACIÓN DE FILAS**

- Al analizar la columna de **employee number**, encontramos 208 valores duplicados 

- Detectamos que la única diferencia entre estos duplicados eran los valores que se encuentran en la columna 'remotework'. Allí detectamos que la equivalencia era 0 = 'No' y 1 = 'Yes'

In [82]:
#identificamos los números de empleado duplicados
# usamos unique para que muestre los valores únicos
duplicados = df_ejemplo[df_ejemplo.duplicated(subset=['employeenumber'], keep=False)]['employeenumber'].unique()
len(duplicados)

#Hay 104 numeros de empleado duplicados más NaN

105

In [83]:
# Almacenamos los números de empleados duplicados en una lista

lista_duplicados2= []

for numero in duplicados:
    lista_duplicados2.append(df_ejemplo[df_ejemplo['employeenumber'] == numero])

In [84]:
# lo visualizamos en un nuevo dataFrame
df_resultado = pd.concat(lista_duplicados2)

# Reseteaamos los índices y los convertimos en una columna

df_resultado.reset_index(inplace=True)
df_resultado.head(6)

,index,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,yearbirth,remotework
0,82,38,No,NaN,119.0,NaN,3.0,3,Life Sciences,307.0,1,male,76,3,3,Sales exECuTIVE,3,Divorced,NaN,9647,0,NaN,12,3.0,3,2,NaN,6,2.0,16,5,13,1985,Yes
1,1474,38,No,NaN,119.0,NaN,3.0,3,Life Sciences,307.0,1,male,76,3,3,SAleS EXeCuTive,3,Divorced,NaN,9647,0,NaN,12,3.0,3,2,NaN,6,2.0,16,5,13,1985,Yes
2,84,55,No,travel_rarely,452.0,NaN,1.0,3,Medical,374.0,4,male,81,3,5,manAger,1,Single,19045.0,18938,0,NaN,14,3.0,3,0,37.0,2,3.0,36,4,13,1968,Yes
3,1476,55,No,travel_rarely,452.0,NaN,1.0,3,Medical,374.0,4,male,81,3,5,MaNAger,1,Single,19045.0,18938,0,NaN,14,3.0,3,0,37.0,2,3.0,36,4,13,1968,Yes
4,89,31,No,NaN,480.0,NaN,NaN,2,NaN,524.0,2,female,31,3,2,MaNufactuRINg dIrecTor,1,Married,4306.0,4156,1,No,12,3.0,2,1,13.0,5,1.0,13,3,12,1992,Yes
5,1481,31,No,NaN,480.0,NaN,7.0,2,NaN,524.0,2,female,31,3,2,maNUFActUrInG dIreCTOr,1,Married,4306.0,4156,1,No,12,3.0,2,1,13.0,5,1.0,13,3,12,1992,Yes


Detectamos que hay 104 filas a eliminar, ya que los duplicados van de dos en dos. 
Eliminamos los índices que están en posición par.

In [85]:
#cogemos los índices de 2 en 2 

indices_a_sacar = df_resultado.iloc[::2, 0]

len(indices_a_sacar)

104

In [86]:
indices_a_sacar

0        82
2        84
4        89
6        90
8       100
       ... 
198    1005
200    1006
202    1007
204    1008
206    1010
Name: index, Length: 104, dtype: int64

In [87]:
# Eliminamos las filas duplicadas a partir de los índices que hemos sacado:

df_ejemplo = df_ejemplo.drop(indices_a_sacar)

In [88]:
df_ejemplo.shape

# Pasamos de 1614 filas a 1510 --> Comprobamos que se han eliminado las 104 filas especificadas.

(1510, 33)

In [89]:
# comprobamos que todos los valores son únicos, y que por lo tanto no hay duplicados.

value_counts = df_ejemplo['employeenumber'].value_counts()
valores_mayor_que_1 = value_counts[value_counts > 1]
valores_mayor_que_1

Series([], Name: count, dtype: int64)

____

# Transformación de datos III

- Homogeneizamos las categorías de las encuestas (que todas vayan de 1 a 4)

- Modificamos los valores de Education para que sean más legibles
- Cambiamos los float a int. Lo miramos cuando gestionamos nulos

- JobLevel, está de 1 a 5. Cambiamos por los siguientes por categorías profesionales:


|Job Level              | Descripción                                       | 
|--------------         |---------------------------------------------------|
| 1 - `Entry`           | Limited or prior experience / Non Exempt / PSS
| 2 - `Intermediate`    | Assignments of moderate scope and complexity / Judgment within defined guidelines / Non Exempt / PSS | 
| 3 - `Experienced`     | In depth understanding of field / Independently performs full range of function / Exempt / PSS    | 
| 4 - `Advanced`        | High degree knowledge of overall field / Specialized knowledge / High degree of autonomy / Specialized knowledge / High degree of autonomy / Lead / Exempt / PSS or MSP | 
| 5 - `Expert`          | Significant impact on policy program / Substantial consequences of success or failure / Scope impacts entire organization (campus) / Exempt / MSP| 




In [90]:
df_ejemplo['jobsatisfaction'].value_counts() #1 a 4
df_ejemplo['stockoptionlevel'].value_counts() # 0 a 3
df_ejemplo['relationshipsatisfaction'].value_counts() # 1 a 4
df_ejemplo['performancerating'].value_counts() # 3.0 y 4.0 
df_ejemplo['jobinvolvement'].value_counts() #1 a 4
df_ejemplo['worklifebalance'].value_counts() #1.0 a 4.0
df_ejemplo['trainingtimeslastyear'].value_counts() # 0 a 6
df_ejemplo['joblevel'].value_counts() # 1 a 5


joblevel
1    555
2    546
3    228
4    111
5     70
Name: count, dtype: int64

In [91]:
datos = {1: 'High school', 2: 'College', 3: 'University', 4:'Master Degree', 5: 'PhD'}

In [92]:
df_ejemplo['education']= df_ejemplo['education'].replace(datos)

In [93]:
categorias = {1: 'Entry', 2: 'Intermediate', 3: 'Experienced', 4: 'Advanced', 5: 'Expert'}

In [94]:
df_ejemplo['joblevel']= df_ejemplo['joblevel'].replace(categorias)

## Transformación de Datos IV

 - Modficamos el valor 'Not Available' por 'Unknown' de la columna 'hourlyrate' 

In [95]:
df_ejemplo['hourlyrate'] = df_ejemplo['hourlyrate'].replace( 'Not Available', 'Unknown')

______
# Guardamos el DataFrame modificado

Cuando los datos están limpios, pasamos el DataFrame a .csv para poder tratar los nulos. Se trabajará en el archivo "orden_gestion_nulos.ipynb"

In [96]:
df_ejemplo.to_csv('hr_limpio_con_nulos.csv')